In [28]:
##############
#K. David Roell CFPB 
#collects row-level and counts of data that fail distribution style quality edts
#requires precomputed bins loaded as bin_df from ../linked_dist_stats/IQR_all.csv
#this file is generated using the IQR cleaning viz script which uses data from the get_linked_dist script
##############
from collections import OrderedDict
import json
import os
import pandas as pd
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from math import sqrt
get_ipython().magic(u'matplotlib inline')
matplotlib.style.use('ggplot')

with open('../quality_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
    edit_sql = json.load(f)
    
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',
}

try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)
    
#read csv of distributions
loanType = 'VA'
bin_df = pd.read_csv('../linked_dist_stats/IQR_{loanType}.csv'.format(loanType=loanType))
bin_df.drop('Unnamed: 0', inplace=True, axis=1)
bin_df.tail()

i'm connected


,name,std,mean,median,max_mult,med_mult,min_mult,q98,q02,q75,q25,iqr,min_q,max_q,count of loans,pct of total loans
15,440 to 470,96.692426,213.916264,197.0,1.060000,0.434783,0.021978,423.60,66.40,276.00,138.00,138.00,-69.000,483.000,721,0.134810
16,470 to 500,105.660545,206.827789,186.0,1.065539,0.387755,0.002083,448.00,48.20,275.00,120.50,154.50,-111.250,506.750,577,0.107885
17,500 to 530,96.834785,206.457500,190.0,0.922000,0.367020,0.077799,421.06,67.96,270.25,130.75,139.50,-78.500,479.500,462,0.086383
18,530 to 560,95.696450,212.987421,199.5,0.863388,0.363802,0.085923,427.32,61.68,274.75,140.00,134.75,-62.125,476.875,371,0.069368
19,561 and up,173.885912,250.343015,200.0,2.194305,0.233021,0.001389,741.18,51.00,299.00,136.00,163.00,-108.500,543.500,2892,0.540736


In [29]:
#get counts based on new distribution method
def get_count_edit_fails(table=None, inc_max=None, inc_min=None, amt_min=None, amt_max=None, name=None, prop_type="'1', '2', '3'", loan_purp="'1','2','3'", loan_type="'1','2','3','4'", occupancy="'1','2','3'", lien="'1','2','3'"):
    """generates a SQL statement to get counts of loans outside of expected distributions"""
    
    #set base selection and table
    base_SQL = """SELECT '{name}' AS bin, {inc_max} AS bin_top, COUNT(amount) FROM {table} """.format(table=table, name=name, inc_max=inc_max)
    
    #set conditions for edit failure
    where_SQL = """WHERE (amount::INT NOT BETWEEN {amt_min} AND {amt_max}) AND
    (income::INT BETWEEN {inc_min} AND {inc_max}) """.format(amt_min=amt_min, amt_max=amt_max, inc_min=inc_min, inc_max=inc_max)
    
    filter_SQL = """AND property_type IN ({prop_type}) AND loan_type IN ({loan_type}) AND 
    loan_purpose IN ({loan_purp}) AND occupancy IN ({occupancy}) AND lien IN ({lien}) 
    """.format(prop_type=prop_type, loan_type=loan_type, loan_purp=loan_purp, occupancy=occupancy, lien=lien)
    #remove NA values from numeric fields
    na_SQL = """AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;"""

    final_SQL = base_SQL + where_SQL + filter_SQL + na_SQL #combine SQL strings to form query

    return final_SQL
    #sum failed rows



In [30]:
#get rows of failed edits with bin label
def get_row_edit_fails(table=None, inc_max=None, inc_min=None, amt_min=None, amt_max=None, name=None, prop_type="'1', '2', '3'", loan_purp="'1','2','3'", loan_type="'1','2','3','4'", occupancy="'1','2','3'", lien="'1','2','3'"):
    """generates a SQL statement to get rows of loans outside of expected distributions"""
    
    #set base selection and table
    base_SQL = """SELECT '{name}' AS bin, {inc_max} AS bin_top, agency, RID, income, amount, sequence FROM {table} """.format(table=table,  name=name, inc_max=inc_max)
    
    #set conditions for edit failure
    where_SQL = """WHERE (amount::INT NOT BETWEEN {amt_min} AND {amt_max}) AND
    (income::INT BETWEEN {inc_min} AND {inc_max}) """.format(amt_min=amt_min, amt_max=amt_max, inc_min=inc_min, inc_max=inc_max)
    
    filter_SQL = """AND property_type IN ({prop_type}) AND loan_type IN ({loan_type}) AND 
    loan_purpose IN ({loan_purp}) AND occupancy IN ({occupancy}) AND lien IN ({lien}) 
    """.format(prop_type=prop_type, loan_type=loan_type, loan_purp=loan_purp, occupancy=occupancy, lien=lien)
    #remove NA values from numeric fields
    na_SQL = """AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;"""

    final_SQL = base_SQL + where_SQL + filter_SQL + na_SQL #combine SQL strings to form query

    return final_SQL
    #sum failed rows



In [31]:
#get edit fails based on distribution method
#this cell retrieves counts of edit fails by year (year must be adjusted manually)

#kwargs are used to set filters for market product
if loanType == 'all':
    loan_type = "'1','2','3','4'"
elif loanType == 'CONV':
    loan_type = "'1'"
elif loanType == 'VA':
    loan_type = "'2'"
elif loanType == 'FHA':
    loan_type = "'3'"
elif loanType == 'RHS':
    loan_type = "'4'"
    
kwargs= {'table':'hmdalar2014',
         'prop_type':"'1'",
        'loan_purp':"'1'",
        'loan_type':loan_type,
        'occupancy':"'1'", 
        'lien':"'1'"}


first = True #set boolean flag to instantiate first instance of dataframe
#FIXME: convet this to a function that takes a function argument for get_count_edit_fails
for index, row in bin_df.iterrows(): #iterate over bins of income from bin_df 
#    print(index)
    #set variable arguments for sql query filters
    kwargs['name'] = bin_df.name.loc[index]

    
    kwargs['inc_min'] = int(kwargs['name'].rsplit(' ', 2)[0])
    if kwargs['name'][-2:] == 'up':
        kwargs['inc_max'] = 9999
    else:
        kwargs['inc_max'] = int(kwargs['name'].rsplit(' ', 2)[2])
    
    kwargs['amt_min'] = int(bin_df.q02.loc[index])
    kwargs['amt_max'] = int(bin_df.q98.loc[index])
    
    sql = get_count_edit_fails(**kwargs) #format query using kwargs dictionary
    print(sql) #check query text
    cur.execute(sql,) #execute SQL
    data_df = pd.DataFrame(cur.fetchall()) #convert results to dataframe
    
    col_names = [desc[0] for desc in cur.description] # get column names from SQL table    
    if len(data_df.columns) == len(col_names): #name dataframe columns if query not empty

        print('setting column names')
        data_df.columns = col_names

    if first == True and len(data_df.columns) > 0: #set master data frame
        dist_df = data_df.copy()
        first = False

    elif first == False and len(data_df.columns) > 0:
        print('merging dataframes')
        dist_df=pd.concat([dist_df, data_df]) #concat new data to master dataframe

    else:
        print('no data returned from query')

    path = '../linked_dist_stats/' #establish data file path
    if not os.path.exists(path):
        os.makedirs(path)
        
    print(dist_df.tail()) #check dataframe contens

#    dist_df['name'] = bin_df.name.loc[index] #set bin ranges as name
dist_df.to_csv(path + 'distribution_edit_fails_{loanType}.csv'.format(loanType=loanType)) #save dataframe to file

SELECT '0 to 20' AS bin, 20 AS bin_top, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 15 AND 210) AND
    (income::INT BETWEEN 0 AND 20) AND property_type IN ('1') AND loan_type IN ('2') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
       bin  bin_top  count
0  0 to 20       20    496
SELECT '20 to 50' AS bin, 50 AS bin_top, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 18 AND 304) AND
    (income::INT BETWEEN 20 AND 50) AND property_type IN ('1') AND loan_type IN ('2') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
merging dataframes
        bin  bin_top  count
0   0 to 20       20    496
0  20 to 50       50    974
SELECT '50 to 80' AS bin, 80 AS bin_top, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 32 AND 412) AND
    

#get rows of edit fails
#get edit fails based on distribution method
#this cell retrieves counts of edit fails by year (year must be adjusted manually)

#kwargs are used to set filters for market product
#this script should be tested with all loan types and property types
kwargs= {'table':'hmdalar2014',
         'prop_type':"'1'",
        'loan_purp':"'1'",
        'loan_type':"'1'",
        'occupancy':"'1'", 
        'lien':"'1'"}


first = True #set boolean flag to instantiate first instance of dataframe

for index, row in bin_df.iterrows(): #iterate over bins of income from bin_df 
#    print(index)
    #set variable arguments for sql query filters
    kwargs['name'] = bin_df.name.loc[index]
    kwargs['inc_min'] = int(kwargs['name'].rsplit(' ', 2)[0])
    if kwargs['name'][-2:] == 'up':
        kwargs['inc_max'] = 9999
    else:
        kwargs['inc_max'] = int(kwargs['name'].rsplit(' ', 2)[2])
    kwargs['amt_min'] = bin_df.q02.loc[index]
    kwargs['amt_max'] = bin_df.q98.loc[index]
    
    sql = get_row_edit_fails(**kwargs) #format query using kwargs dictionary
    print(sql) #check query text
    cur.execute(sql,) #execute SQL
    data_df = pd.DataFrame(cur.fetchall()) #convert results to dataframe
    
    col_names = [desc[0] for desc in cur.description] # get column names from SQL table
    
    if len(data_df.columns) == len(col_names): #name dataframe columns if query not empty
        print('setting column names')
        data_df.columns = col_names

    if first == True and len(data_df.columns) > 0: #set master data frame
        dist_df = data_df.copy()
        first = False

    elif first == False and len(data_df.columns) > 0:
        print('merging dataframes')
        dist_df=pd.concat([dist_df, data_df]) #concat new data to master dataframe

    else:
        print('no data returned from query')

    path = '../linked_dist_stats/' #establish data file path
    if not os.path.exists(path):
        os.makedirs(path)
        
    print(dist_df.tail()) #check dataframe contens

    dist_df['name'] = bin_df.name.loc[index] #set bin ranges as name
    dist_df.to_csv(path + 'distribution_row_edit_fails{loanType}.csv'.format(loanType=loanType)) #save dataframe to file

In [32]:
#compress tail bins for counts df into one 600+ income
fail_counts_df = pd.read_csv(path + 'distribution_edit_fails_{loanType}.csv'.format(loanType=loanType))

In [33]:
fail_counts_df.head()

,Unnamed: 0,bin,bin_top,count
0,0,0 to 20,20,496
1,0,20 to 50,50,974
2,0,50 to 80,80,1166
3,0,80 to 110,110,1287
4,0,110 to 140,140,920
